In [174]:
import openmc
import openmc.mgxs
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas
import h5py
%matplotlib inline

In [175]:
import os

In [176]:
os.environ['PATH'] += r':/home/jae123p/miniconda3/envs/openmc-env/bin'

%env OPENMC_CROSS_SECTIONS=/home/jae123p/endfb80/endfb-viii.0-hdf5/cross_sections.xml

env: OPENMC_CROSS_SECTIONS=/home/jae123p/endfb80/endfb-viii.0-hdf5/cross_sections.xml


In [177]:
model = openmc.Model()

In [179]:
oralloy = openmc.Material(name="Oralloy")
oralloy.add_nuclide('U234', 4.9210e-04)
oralloy.add_nuclide('U235', 4.4917e-02)
oralloy.add_nuclide('U238', 2.5993e-03)

nickel = openmc.Material(name = "nickel")
nickel.set_density('sum')
nickel.add_element('Ni', 9.1322e-02)

materials = openmc.Materials([oralloy, nickel])
materials.export_to_xml('HEU_MET_FAST-003-CASE-1.xml')
model.materials = openmc.Materials([oralloy, nickel])

In [180]:
#Geometry
inner_radius = 6.4627
outer_radius = 26.7827

oralloy_sphere = openmc.Sphere(0, 0, 0, r=inner_radius)
nickel_sphere = openmc.Sphere(0, 0, 0, r=outer_radius, boundary_type = 'vacuum')


In [181]:
#Regions
or_region = -oralloy_sphere
nickel_region = +oralloy_sphere & - nickel_sphere

#Cells
or_cell = openmc.Cell(name='or', fill = oralloy, region = or_region)
nickel_cell  = openmc.Cell(name='nickel', fill = nickel, region = nickel_region)

geometry = openmc.Geometry([or_cell, nickel_cell])
geometry.export_to_xml()

In [182]:
root_universe = openmc.Universe(cells = (or_cell, nickel_cell))
model.geometry = openmc.Geometry(root_universe)
cell = openmc.Cell(name = 'cell',fill = root_universe)

In [183]:
#Settings 
settings = openmc.Settings()
settings.batches = 3000
settings.inactive = 20
settings.particles = 10000
settings.runmode = 'eigenvalue'

#Define Source
lower_left = (-1, -1, -1)
upper_right = (1, 1, 1)
uniform_dist = openmc.stats.Box(lower_left, upper_right)
#energy_dist = openmc.stats.Uniform(a=0.0e6,b=20.0e6)
settings.source = openmc.IndependentSource(space=uniform_dist)
settings.export_to_xml()
model.settings = settings


settings.output = {'tallies':False}

In [184]:
my_path = './mgxs_' + cell.name

In [185]:
lanl30_edges= [
    1.00e-5,  # Thermal cutoff (lower bound)
    1.00e+0, 5.00e+0, 1.00e+1, 5.00e+1, 1.00e+2, 3.00e+2, 5.00e+2, 1.00e+3, 2.00e+3,
    3.00e+3, 5.00e+3, 1.00e+4, 2.00e+4, 3.00e+4, 5.00e+4, 8.00e+4, 1.00e+5, 1.50e+5,
    2.00e+5, 3.00e+5, 4.00e+5, 6.00e+5, 8.00e+5, 1.00e+6, 1.50e+6, 2.00e+6, 3.00e+6,
    5.00e+6, 1.00e+7, 2.00e+7  # Upper bound (fast neutrons)
]

groups = openmc.mgxs.EnergyGroups(group_edges=lanl30_edges)

In [186]:
# Initialize MGXS Library
mgxs_lib = openmc.mgxs.Library(model.geometry)

# Add the chosen group structure
mgxs_lib.energy_groups = groups

# Scatttering Format and Legendre Order
mgxs_lib.scatter_format = "legendre"
mgxs_lib.legendre_order = 3

# Specify multi-group cross-section types to compute
mgxs_lib.mgxs_types = ['total']
# set uncertainty goal
mgxs_lib.tally_trigger = openmc.Trigger('std_dev', 1e-4)

# Compute cross sections on a nuclide-by-nuclide basis
mgxs_lib.by_nuclide = False

# Specify a "cell" domain type for the cross section tally filters
mgxs_lib.domain_type = 'material'

# Specify the cell domains over which to compute multi-group cross sections
#mgxs_lib.domains = model.geometry.get_all_cells().values()

# Construct all tallies needed for the multi-group cross section library
mgxs_lib.build_library()

tallies = openmc.Tallies()
mgxs_lib.add_to_tallies_file(tallies, merge=True)
model.tallies = tallies

/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)


In [187]:
sp_file = model.run()

[LAPTOP-H8MU6HJU:10102] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.LAPTOP-H8MU6HJU.1000/jf.0/365821952/shared_mem_cuda_pool.LAPTOP-H8MU6HJU could be created.
[LAPTOP-H8MU6HJU:10102] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%

In [21]:
# Load the last statepoint file
sp = openmc.StatePoint(sp_file)

In [22]:
# Initialize MGXS Library with OpenMC statepoint data
mgxs_lib.load_from_statepoint(sp)

In [24]:
# Store the cross section data in an "mgxs/mgxs.h5" HDF5 binary file
mgxs_lib.build_hdf5_store(filename='openmc.h5')

In [25]:
sp.close()